In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import tweepy
#import GetOldTweets3 as got
import re
from collections import Counter
from string import punctuation
from tweepy import OAuthHandler
import json
import os
from dotenv import load_dotenv
import time
import csv

### Este jupyternotebook es para iniciar  la extraccion de datos y poder iniciar todo el proceso.

In [15]:
load_dotenv()


CONSUMER_KEY = os.getenv("consumer_key")
CONSUMER_SECRET = os.getenv("consumer_secret")
ACCESS_TOKEN = os.getenv("access_token")
ACCESS_SECRET = os.getenv("access_token_secret")

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

user=api.me()
print(user.name,",", user.screen_name)

卡西亚 , nadie_me_tiene


In [22]:
textblob, spicy (NN), word to vect, nltk#api.update_status("Preparandome para el trabajo final de IronHack")
#https://twitter.com/nadie_me_tiene/status/1313883047469686790

TweepError: [{'code': 187, 'message': 'Status is a duplicate.'}]

In [28]:
start_time = time.time()

max_id = -1000000000000000000000000000000000
tweetsPerQry = 1000  
search_words = '(@JoeBiden OR @DonaldTrump OR Donald Trump OR Joe Biden)'
date_since = "2020-09-09" 

# Collect tweets
# open/create a csv file to append data
csvFile = open(search_words + '_data3.csv', 'w', encoding='utf-8')

# use csv Writer
csvWriter = csv.writer(csvFile)

# get data from twitter
tweet_num = 0
for tweet in tweepy.Cursor(
                        api.search,
                        q=search_words + ' -RT',
                        count=tweetsPerQry,
                        lang="en", 
                        since=date_since, max_id=str(max_id - 1)).items(9999999999):
    if tweet.place is not None:
        try:
            # not entirely necessary but you can inspect what is being written to file
            print ('tweet number: {}'.format(tweet_num), tweet.text, tweet.place.full_name)
            # write data to csv
            csvWriter.writerow([tweet.created_at,
                                tweet.user.screen_name,
                                tweet.user.location,
                                tweet.user.followers_count,
                                tweet.user.friends_count,
                                tweet.text,
                                tweet.place.bounding_box.coordinates,
                                tweet.place.full_name,
                                tweet.place.country,
                                tweet.place.country_code,
                                tweet.place.place_type])
            tweet_num += 1
            
        except :
            continue

print("--- %s seconds ---" % (time.time() - start_time))


tweet number: 0 @myladydisdain @bevdevwarrior @JoeBiden Biden has been doing Town Hall meetings all along..
He has a lot of GOOD th… https://t.co/Nnp2JVEZIM Bunker Hill, OR
tweet number: 1 @JoeBiden has stated numerous times that he would institute a mask mandate once in office.

If masks work so well,… https://t.co/KJ6qZH3hEE Indiana, USA
tweet number: 2 @JoeBiden has stated numerous times that he would institute a mask mandate once in iffice.

If masks work so well,… https://t.co/ZgPIkhPVGJ Indiana, USA
tweet number: 3 Indeed. Which is one of many reasons I could never vote for either Joe Biden nor Donald Trump. https://t.co/vfKWRHR28y Chicago, IL
tweet number: 4 Donald Trump literally tested positive for Covid last week and is now saying the idea of him having to debate with… https://t.co/A7G5poBU5z Portsmouth, NH
tweet number: 5 @RealMirandaKhan @RealDrGina @realDonaldTrump @JoeBiden @RealAVNews So - what I see from Biden team - teleprompters… https://t.co/Atzr76xmby North Windham,

tweet number: 59 @oliteheart @sandstock 9 days since the presidential  debate, Joe Biden still won't answer the if he will pack the… https://t.co/ewM2fYZHta Indianapolis, IN
tweet number: 60 @callynt 9 days since the presidential  debate, Joe Biden still won't answer the if he will pack the court.... 

It… https://t.co/u0xcMwiTOC Indianapolis, IN
tweet number: 61 @KatiePavlich @Paulhardingjr 9 days since the presidential  debate, Joe Biden still won't answer the if he will pac… https://t.co/8OiAuEAnbL Indianapolis, IN
tweet number: 62 @sirhottest 9 days since the presidential  debate, Joe Biden still won't answer the if he will pack the court....… https://t.co/1xDobTGpGt Indianapolis, IN
tweet number: 63 @CopJrCliff 9 days since the presidential  debate, Joe Biden still won't answer the if he will pack the court....… https://t.co/i1j2o8gsuo Indianapolis, IN
tweet number: 64 @RipTorn18 9 days since the presidential  debate, Joe Biden still won't answer the if he will pack the court....…

tweet number: 111 @GlutenFreeGanja @HTTRWahoowa @JoeBiden Trump isn’t doing anything illegal. He is utilizing business tax deductions… https://t.co/seykFyw9QF Dakota Dunes, SD
tweet number: 112 @JoeBiden Cubans for Joe Biden Miami, FL
tweet number: 113 @corvusglaive19 @toppr0sp3ct @ColinCowherd Has Joe Biden given anyone a reason to vote for him other than him not being Donald Trump? Lake Providence, LA
tweet number: 114 This has been a weird ass 2020 presidential debate lately huh like wtf in mike pence eye 🤔🤨🧐

Donald Biden COVID CO… https://t.co/3lduwFAk1X Greenville, MS
tweet number: 115 Nearly 500 generals, admirals endorse Joe Biden blast Donald Trump
https://t.co/2CZb2xQPDT Los Angeles, CA
tweet number: 116 @JoeBiden Lying Joe Biden Davie, FL
tweet number: 117 @JoeBiden A Biden-Harris administration will make Trump pay his taxes! Parma, OH
tweet number: 118 @MittRomney @UtahDebateCom @UUtah How do you as a LDS. choose who to lead?..it must be so difficult.i am LDS, live… https:/

tweet number: 164 Donald Trump does not respect US military members or Veterans.  Biden’s son actually served! He gets it! #VPDebate Minneapolis, MN
tweet number: 165 Donald Trump believes that Mike Pence’s son is a loser. Joe Biden and Kamala Harris don't. #VPDebate Portland, OR
tweet number: 166 @Jennifer2Flower @ravenlenore01 @KamalaHarris @JoeBiden I realize you're getting very used to lying from Trump but… https://t.co/nxLWKCe4dS Colorado, USA
tweet number: 167 Joe Biden and Kamala Harris do not have a platform for their campaign. They don’t have plans. Kamala’s answer to ev… https://t.co/CUbIB87ker Rensselaer, IN
tweet number: 168 i wld pay to hear Pence call trump just Donald .... like Kamala call Biden Joe when she talks abt him @KamalaHarris @VP Matthews, NC
tweet number: 169 @Mike_Pence :: Joe Biden lie person. american is saver.Trump never hate the caption.@SenKamalaHarris : personal par… https://t.co/BjidGq1jad Pasar Kliwon, Indonesia
tweet number: 170 So pretty much Donald

tweet number: 218 @JoeBiden @Araya_Hope____ @BarackObama @MichelleObama @DrBiden You need to win this election sir joe Biden so that… https://t.co/uKaFxW2svT Port Harcourt, Nigeria
tweet number: 219 im not voting for joe biden unless he promises everyone a $2,000 stimulus check upon being elected . @JoeBiden… https://t.co/dWdWRnmvDn Irvine, CA
tweet number: 220 I live in Orange County, Florida (one of the most important counties for @JoeBiden). I literally can’t believe how… https://t.co/IfwRIabViA Ocoee, FL
tweet number: 221 Joe Biden&gt;&gt;&gt;&gt;&gt;&gt; Donald Trump Fort Valley, GA
tweet number: 222 Joe Biden Caught Red-Handed Misleading Americans About the Number of People Given Clemency While He Was VP via… https://t.co/VdjbEBZe1b Los Angeles, CA
tweet number: 223 Vote for biden if you have seen allá the chaos and how racist donald trump is, he broke sustainable schemes for hum… https://t.co/Eg9hIEiplV Guatemala
tweet number: 224 @NeedMoreLuv @JoeBiden This really make me sick.

...

tweet number: 2914 Biden won trump was a lying bully and supports the proud boys and  white supremacist.  @GOP Gang of Putin… https://t.co/fEvaCEp4Z8 Colorado, USA
tweet number: 2915 this breaks my heart how trump makes fun of someone who has died and served the military. this is not okay. fuck do… https://t.co/FPRKKLWY2m Salinas, CA
tweet number: 2916 @DrBiden Thank you Dr. Jill Biden. At the end of the debate last night you showed us what a real wife is &amp; what a r… https://t.co/sKBUOqvWkj Cocoa, FL
tweet number: 2917 @JoeBiden Joe Biden is WINNING 

Save this Tweet.

🇺🇲🇮🇳💕 Faridabad, India
tweet number: 2918 @JoeBiden Great Job yesterday. 
Vice President Joe Biden 
Awesome‼ ✌🌟❤ Fincastle, KY
tweet number: 2919 @realDonaldTrump Joe Biden had a wire @JoeBiden @FoxNews @CNN @cnnbrk @ABC @NBCNews @9NEWS @CBSNews https://t.co/HBsfwKguNH Northglenn, CO
tweet number: 2920 Steelers Wheel..
https://t.co/TcV5mR7j2C Leeds, England
tweet number: 2921 The mediator, Chris Wallace, fought with 

tweet number: 2965 “BOTH DONALD TRUMP AND JOE BIDEN DID SO EQUALLY BAD” and other lies you need to tell yourself to vote for Donald Tr… https://t.co/yrhbGesRhb Philadelphia, PA
tweet number: 2966 @Barnesy19 @JoeBiden Sorry Barnes but I think Biden had less food on his face tonight. Trump needs to up his game for round two. Ilford, London
tweet number: 2967 Debates 2020: Highlights from the first presidential debate between Donald Trump and Joe Biden #Debates2020 https://t.co/t1FkIJwF92 Montréal, Québec
tweet number: 2968 @JoeNBC Nancy Pelosi said it earlier. Joe Biden should not debate Donald Trump, she predicted what happened tonight… https://t.co/BfIjAs8Uje Toronto, Ontario
tweet number: 2969 @BlackVoices4DJT @TeamTrump @JoeBiden Biden has lost his mind I guess. Trump clobbered him. Houston, TX
tweet number: 2970 Imho I still like the fact that Joe Biden called Donald Trump a clown. I have it on good sources it can be a thing. Hillsboro, OR
tweet number: 2971 @JoeBiden every tweet is

tweet number: 3016 @JoeBiden I will not vote this time but I agree Trump is the worse President we ever had Biden is a warmonger hope… https://t.co/Quzt0IGnYj Paradise, NV
tweet number: 3017 @DaykaYasmine @JoeBiden U obviously haven’t done ur homework.Research Biden’s career as a politician.He’s a corrupt… https://t.co/y3SdC3rWgR Pennsylvania, USA
tweet number: 3018 You all know that Donald Trump is scared shitless of Joe Biden, right? Wellesley, MA
tweet number: 3019 If you still support Donald Trump after that unpresidential and childish debate IDK what to say to y’all I’m voting… https://t.co/g63XvDXE69 Taunton, MA
tweet number: 3020 Text TRUMP to 88022! I stand with Donald Trump! https://t.co/LqgfJzP4E1 Sunrise, FL
tweet number: 3021 We'll have to see what the polls say. I wanted Joe Biden to be more abrasive with Donald Trump but maybe the undeci… https://t.co/GFss9BdEUH San Diego, CA
tweet number: 3022 Donald Trump was a combative ridiculously incoherent mess on the debate stage 

Rate limit reached. Sleeping for: 755


KeyboardInterrupt: 

In [16]:
#Defino las columnas que voy a utilizar y paso la informacion a un CSV
col_names=['date','user_name','user_loc','followers','friends','message','bbox_coords',\
           'full_name','country','country_code','place_type']
df2= pd.read_csv('(@JoeBiden OR @DonaldTrump OR Donald Trump OR Joe Biden)_data3.csv', names=col_names)
df2
#Tras un exitoso web scrapping consigo rascar 2300 tweets

,date,user_name,user_loc,followers,friends,message,bbox_coords,full_name,country,country_code,place_type
0,2020-10-08 13:52:54,littlegrandma63,Oregon,4895,4925,@myladydisdain @bevdevwarrior @JoeBiden Biden ...,"[[[-124.22592, 43.341006], [-124.194494, 43.34...","Bunker Hill, OR",United States,US,city
1,2020-10-08 13:39:32,tempio_joe,United States 🇺🇸,4655,4982,@JoeBiden has stated numerous times that he wo...,"[[[-88.097892, 37.771743], [-84.78458, 37.7717...","Indiana, USA",United States,US,admin
2,2020-10-08 13:38:21,tempio_joe,United States 🇺🇸,4655,4982,@JoeBiden has stated numerous times that he wo...,"[[[-88.097892, 37.771743], [-84.78458, 37.7717...","Indiana, USA",United States,US,admin
3,2020-10-08 13:36:23,EliasCepeda,Chicago,2242,2102,Indeed. Which is one of many reasons I could n...,"[[[-87.940033, 41.644102], [-87.523993, 41.644...","Chicago, IL",United States,US,city
4,2020-10-08 13:35:57,xoahriellee,new england,171,256,Donald Trump literally tested positive for Cov...,"[[[-70.820242, 43.0136731], [-70.737324, 43.01...","Portsmouth, NH",United States,US,city
...,...,...,...,...,...,...,...,...,...,...,...
3003,2020-09-30 05:26:10,omollo_o,Nairobi,760,133,@Belive_Kinuthia @JoeBiden @realDonaldTrump Di...,"[[[36.6645733, -1.3890532], [37.0626672, -1.38...","Nairobi, Kenya",Kenya,KE,city
3004,2020-09-30 05:25:19,trump_friends,"Bexley, London",2534,4886,Donald J. Trump\n\nJoe Biden could not name a ...,"[[[0.045148, 51.423888], [0.1335005, 51.423888...","Eltham, London",United Kingdom,GB,city
3005,2020-09-30 05:23:57,kaos19juice,"Newmarket, Ontario",241,801,@blackknight_n @SchwiftyEmmy @TheRealBKB @JoeB...,"[[[-79.712044, 44.012924], [-79.511397, 44.012...","Bradford West Gwillimbury, Ontario",Canada,CA,city
3006,2020-09-30 05:23:24,trump_friends,"Bexley, London",2534,4886,"Members of the Trump family (L-R) Eric Trump, ...","[[[0.045148, 51.423888], [0.1335005, 51.423888...","Eltham, London",United Kingdom,GB,city


In [17]:
df2.isna().sum()

date              0
user_name         0
user_loc        642
followers         0
friends           0
message           0
bbox_coords       0
full_name         0
country           6
country_code      6
place_type        0
dtype: int64

In [18]:
df2.country_code.value_counts()

US    2621
GB      87
CA      78
AU      26
IN      24
      ... 
NE       1
TZ       1
BD       1
IS       1
GN       1
Name: country_code, Length: 64, dtype: int64

In [19]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3008 entries, 0 to 3007
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          3008 non-null   object
 1   user_name     3008 non-null   object
 2   user_loc      2366 non-null   object
 3   followers     3008 non-null   int64 
 4   friends       3008 non-null   int64 
 5   message       3008 non-null   object
 6   bbox_coords   3008 non-null   object
 7   full_name     3008 non-null   object
 8   country       3002 non-null   object
 9   country_code  3002 non-null   object
 10  place_type    3008 non-null   object
dtypes: int64(2), object(9)
memory usage: 258.6+ KB


In [20]:
df2['date'] = pd.to_datetime(df2['date'])

In [22]:
print(df2.date.min())
print(df2.date.max())

2020-09-30 05:23:10
2020-10-08 13:52:54


In [56]:
print(df2.user_name.value_counts())


lroerig1255        92
Walker1world       44
ErinMH90           20
CuttemUp           16
AKkevi7            15
                   ..
JDAcquaviva2        1
MasterMattYoung     1
pgliddon            1
JPFickett           1
Kevyn3899           1
Name: user_name, Length: 2113, dtype: int64


In [23]:
df2.describe()

,followers,friends
count,3008.000000,3008.000000
mean,3469.012965,2720.625997
std,19613.470101,17293.651954
min,0.000000,0.000000
25%,74.000000,229.750000
50%,382.500000,672.500000
75%,1691.000000,2202.000000
max,405576.000000,443136.000000


In [24]:
def geo_mean(x):
    y = json.loads(x)
    y = np.asarray(y)
    mean_geoloc = np.add(np.add((y[:][0][0]),(y[:][0][1]))/2,\
                         (np.add((y[:][0][2]),(y[:][0][3]))/2))/2    
    return np.around(mean_geoloc,decimals=6)
#Uso una funcion para transformar las bounding box(cuadros delimitadores) en coordenadas geograficas (llevar a funcpy)

In [58]:
df2['geo_code'] = df2['bbox_coords'].apply(geo_mean)
df2 = df2.drop(['bbox_coords','place_type','followers','friends'],axis=1)
df2.head()
#Aplico la funcion a la columna, elimino columnas redundantes como el tipo de lugar desde donse tuitea, el numero de seguidores o el numero de amiwis que tienen
#

,date,user_name,user_loc,message,full_name,country,country_code,geo_code
0,2020-10-08 13:52:54,littlegrandma63,Oregon,@myladydisdain @bevdevwarrior @JoeBiden Biden ...,"Bunker Hill, OR",United States,US,"[-124.210207, 43.351643]"
1,2020-10-08 13:39:32,tempio_joe,United States 🇺🇸,@JoeBiden has stated numerous times that he wo...,"Indiana, USA",United States,US,"[-86.441236, 39.766555]"
2,2020-10-08 13:38:21,tempio_joe,United States 🇺🇸,@JoeBiden has stated numerous times that he wo...,"Indiana, USA",United States,US,"[-86.441236, 39.766555]"
3,2020-10-08 13:36:23,EliasCepeda,Chicago,Indeed. Which is one of many reasons I could n...,"Chicago, IL",United States,US,"[-87.732013, 41.833584]"
4,2020-10-08 13:35:57,xoahriellee,new england,Donald Trump literally tested positive for Cov...,"Portsmouth, NH",United States,US,"[-70.778783, 43.056115]"
